In [1]:
from pyspark import SparkContext
import pandas as pd

In [2]:
sc = SparkContext()

In [3]:
import re
import csv
from datetime import datetime
from collections import namedtuple

%matplotlib inline
import matplotlib.pyplot as plt


# pip install workalendar
# https://github.com/novafloss/workalendar
from workalendar.europe import Italy

In [4]:
cal = Italy()

def def_is_holiday(dt_obj):
    if(cal.is_working_day(dt_obj)):
        return 0
    else:
        return 1

In [5]:
row = namedtuple("prodotto", ['zona','area','sottoarea','categoria','timestamp','giorno_della_settimana', 'giorno_del_mese',
                              'giorno_dell_anno', 'mese', 'anno','festivo','vendite'])

In [6]:
def do_parse(x):
    # Oggetto Datetime
    dt_obj = datetime.strptime(str(x[4]), '"%Y-%m-%d"')
    
    return row(
        zona = int(re.sub("[^0-9]", "", str(x[0]))),
        area = int(re.sub("[^0-9]", "", x[1])),
        sottoarea = int(re.sub("[^0-9]", "", x[2])),
        categoria = int(re.sub("[^0-9]", "", x[3])),
        timestamp = int(dt_obj.strftime("%s")),
        giorno_della_settimana = int(dt_obj.weekday()),
        giorno_del_mese = int(dt_obj.weekday()),
        giorno_dell_anno = int(dt_obj.timetuple().tm_yday),
        mese = int(dt_obj.strftime("%m")),
        anno = int(dt_obj.strftime("%Y")),
        festivo = def_is_holiday(dt_obj),
        vendite = int(x[5])
        )

def parse(sc,filepath):
    rawRdd = sc.textFile(filepath)
    
    header = rawRdd.first()

    # Togliamo l'header
    rawRdd = rawRdd.filter(lambda x: x != header)

    # Togliamo le righe vuote
    rawRdd = rawRdd.filter(lambda x: x != "")
    
    return rawRdd.map(lambda x: x.split(",")).map(lambda x: do_parse(x))

In [7]:
rdd = parse(sc,"dataset/dataset_polimi.csv")

In [8]:
info_sottoaree = rdd.map(lambda x: (x.sottoarea,(x.zona,x.area))).distinct().collect()

In [9]:
dict_sottoaree = dict()
for x in info_sottoaree:
    dict_sottoaree[x[0]] = x[1]

In [10]:
row_test = namedtuple("prodotto", ['zona','area','sottoarea','categoria','timestamp','giorno_della_settimana', 'giorno_del_mese',
                              'giorno_dell_anno', 'mese', 'anno','festivo'])
def do_test_parse(x):
    # Oggetto Datetime
    dt_obj = datetime.strptime(str(x[2]), '%d-%m-%Y')
    sotto_area = int(re.sub("[^0-9]", "", x[0]))
    return row_test(
        zona = dict_sottoaree[sotto_area][0],
        area = dict_sottoaree[sotto_area][1],
        sottoarea = sotto_area,
        categoria = int(re.sub("[^0-9]", "", x[1])),
        timestamp = int(dt_obj.strftime("%s")),
        giorno_della_settimana = int(dt_obj.weekday()),
        giorno_del_mese = int(dt_obj.weekday()),
        giorno_dell_anno = int(dt_obj.timetuple().tm_yday),
        mese = int(dt_obj.strftime("%m")),
        anno = int(dt_obj.strftime("%Y")),
        festivo = def_is_holiday(dt_obj),
        )
def test_parse(sc,filepath):
    rawRdd = sc.textFile(filepath)
    
    header = rawRdd.first()

    # Togliamo l'header
    rawRdd = rawRdd.filter(lambda x: x != header)

    # Togliamo le righe vuote
    rawRdd = rawRdd.filter(lambda x: x != "")
    
    return rawRdd.map(lambda x: x.split("\t")).map(lambda x: do_test_parse(x))

In [12]:
test_rdd = test_parse(sc,"dataset/output/test_file.csv")
test_rdd.take(1)

[prodotto(zona=10, area=26, sottoarea=67, categoria=1, timestamp=1463702400, giorno_della_settimana=4, giorno_del_mese=4, giorno_dell_anno=141, mese=5, anno=2016, festivo=0)]

[prodotto(zona=1, area=1, sottoarea=5, categoria=1, timestamp=1388530800, giorno_della_settimana=2, giorno_del_mese=2, giorno_dell_anno=1, mese=1, anno=2014, festivo=1, vendite=0)]

# Pandas
if you cannot stay without it

In [17]:
locations = pd.read_csv('dataset/output/gps_processed.csv',index_col=0)

In [18]:
test_rdd_list = test_rdd.collect()

In [19]:
df = pd.DataFrame.from_records(test_rdd_list,columns = ['zona','area','sottoarea','categoria','timestamp','giorno_della_settimana', 'giorno_del_mese',
                              'giorno_dell_anno', 'mese', 'anno','festivo'])

In [20]:
# Mergiamo le nuove feature e binarizziamo le regioni e zona

df = df.merge(locations[
        ['sottoarea','regione','zona_italia','isola','abitanti','min_distanza','densita','indice_istruzione','reddito_medio','concentraz_ricchezza']
    ], on='sottoarea')

df = pd.get_dummies(df, columns={'regione'} )
df = pd.get_dummies(df, columns={'zona_italia'} )

In [22]:
df.to_csv('dataset/output/test_with_features.csv',sep = ',',index = False)